In [114]:
from bs4 import BeautifulSoup
import requests, yaml, pytz
import datetime as dt
import pandas as pd

In [115]:
efc_url = "https://www.efinancialcareers.co.uk/jobs/in-United-Kingdom"

url = efc_url

In [116]:
def parser():
    headers = headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}
    content = requests.get(url, headers).content
    soup = BeautifulSoup(content, "html.parser")

    return soup

In [117]:
def job_card():
    soup = parser()
    
    efc_jobcard = soup.find('div', {'class': 'd-flex flex-column jobs-container ng-star-inserted'}).find_all('efc-job-card')

    card_details = [card.find('efc-card-details') for card in efc_jobcard]
    card_footer = [card.find('efc-card-footer') for card in efc_jobcard]
        
    return card_details, card_footer

In [118]:
def first_page():
    jobcard = job_card()
    card_det, card_ftr = jobcard[0], jobcard[1]

    job_title, company, job_link, location, position_type, work_arrangement, salary = [], [], [], [], [], [], []
    job_dct = {}

    for item in card_det:
        job_title.append(item.find('a').get('title'))
        company.append(item.find_all('div')[3].text.lstrip().rstrip())
        job_link.append(item.find('a').get('href'))
        location.append(item.find('span' , {'class': 'dot-divider'}).text)
        position_type.append(item.find('span' , {'class': 'dot-divider ng-star-inserted'}).text)
    
    for item in card_ftr:
        ln1 = len(item.find_all('div')[1])
        ln2 = len(item.find_all('div')[1].find_all('span'))
    
        if ln1 == 6 and ln2 == 2:
            work_arrangement.append(item.find_all('div')[1].find_all('span')[0].text)
            salary.append(item.find_all('div')[1].find_all('span')[1].text)
        elif (ln1 == 5 and ln2 == 1) or (ln1 == 7 and ln2 == 2):
            work_arrangement.append(None)
            salary.append(item.find_all('div')[1].find_all('span')[0].text)
    
    job_dct = {"Position": job_title, "Company": company, "Link": job_link, "Location": location, "Position Type": position_type,
                "Work Arrangement": work_arrangement, "Salary": salary}
    final_df = pd.DataFrame(job_dct)

    return final_df

In [119]:
def first_page_test():
    jobcard = job_card()
    card_det, card_ftr = jobcard[0], jobcard[1]

    job_title, company, job_link, location, position_type, work_arrangement, salary = [], [], [], [], [], [], []
    job_dct = {}

    for item in card_det:
        job_title.append(item.find('a').get('title'))
    
    return job_title

In [120]:
test = first_page_test()

print(test)

['People Operations Associate -12 month FTC', 'Technical Project Manager', 'Risk Assessor (Servicing Oversight)', 'Servicing Oversight Analyst', 'Technology Solutions Architect', 'Cyber Security Analyst', 'Crypto Validator Deployment and Operation', 'FX Sales - VP/Director', 'Risk Analyst', 'Information security analyst', 'Head of Internal Audit - Commodities and Global Markets Lead', 'Candidate Management Advisor', 'Senior Software Engineer (Python)', 'Financial Crime Risk Advisor, Senior Manager', 'Infrastructure Debt Portfolio Manager']
